In [1]:
import shutil,glob

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

# Other Python Packages
import numpy as np
import pandas as pd
import string
import re
from nltk.stem import SnowballStemmer
from string import punctuation
import sklearn


#For Reading data from Input files into a training_input file

outfilename="data/train_input.txt"
outfilename2="data/train_labels.txt"
with open(outfilename, 'a',encoding="utf8") as outfile:
    for filename in glob.glob('Data/aclImdb/train/pos/*.txt'):
            if filename == outfilename:
            # don't want to copy the output into the output
                continue
            with open(filename, 'r',encoding="utf8") as readfile:
                outfile.write(readfile.read() + "\n")            
            f=open(outfilename2,'a',encoding="utf8")
            f.write("positive\n")
            f.close()

            
outfilename="data/train_input.txt"
outfilename2="data/train_labels.txt"
with open(outfilename, 'a',encoding="utf8") as outfile:
    for filename in glob.glob('Data/aclImdb/train/neg/*.txt'):
            if filename == outfilename:
            # don't want to copy the output into the output
                continue
            with open(filename, 'r',encoding="utf8") as readfile:
                outfile.write(readfile.read() + "\n")            
            f=open(outfilename2,'a',encoding="utf8")
            f.write("negative\n")
            f.close()
            

            
#For Reading data from Input files into a testing_input file


outfilename="data/test_input.txt"
outfilename2="data/test_labels.txt"
with open(outfilename,'a',encoding="utf8") as outfile:
    for filename in glob.glob('Data/aclImdb/test/pos/*.txt'):
            if filename == outfilename:
            # don't want to copy the output into the output
                continue
            with open(filename, 'r',encoding="utf8") as readfile:
                outfile.write(readfile.read() + "\n")            
            f=open(outfilename2,'a',encoding="utf8")
            f.write("positive\n")
            f.close()
            

            
outfilename="data/test_input.txt"
outfilename2="data/test_labels.txt"
with open(outfilename, 'a',encoding="utf8") as outfile:
    for filename in glob.glob('Data/aclImdb/test/neg/*.txt'):
            if filename == outfilename:
            # don't want to copy the output into the output
                continue
            with open(filename, 'r',encoding="utf8") as readfile:
                outfile.write(readfile.read() + "\n")            
            f=open(outfilename2,'a',encoding="utf8")
            f.write("negative\n")
            f.close()
            

            
#Reading all files separately into DataFrames

df = pd.read_csv('Data/train_input.txt',sep="\n",names = ['text'],header=None, error_bad_lines=False)

df1 = pd.read_csv('Data/train_labels.txt',names = ['label'],sep="\n",header=None, error_bad_lines=False)

df2 = pd.read_csv('Data/test_input.txt',sep="\n",names = ['text'],header=None, error_bad_lines=False)

df3 = pd.read_csv('Data/test_labels.txt',names = ['label'],sep="\n",header=None, error_bad_lines=False)


df= df.dropna()
df = df[df.text.apply(lambda x: x !="")]

df2= df2.dropna()
df2 = df2[df2.text.apply(lambda x: x !="")]


#Shuffling training dataset
train_df=pd.concat([df, df1], axis=1)
train_df = sklearn.utils.shuffle(train_df).reset_index(drop=True)

#Shuffling testing dataset
test_df=pd.concat([df2, df3], axis=1)
test_df = sklearn.utils.shuffle(test_df).reset_index(drop=True)

#One hot encoding of labels
train_labels = train_df['label'].map(lambda x : 1 if x == "positive" else 0)
test_labels = test_df['label'].map(lambda x : 1 if x == "positive" else 0)


#Removing special characters,cleaning and stemming
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    
    text = [w for w in text if len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text
# apply the above function to df['text']
train_df['text'] = train_df['text'].map(lambda x: clean_text(x))
test_df['text'] = test_df['text'].map(lambda x: clean_text(x))

vocabulary_size = 20000
tokenizer = (num_words= vocabulary_size)

#tokenize train data
tokenizer.fit_on_texts(train_df['text'])
sequences = tokenizer.texts_to_sequences(train_df['text'])
data = pad_sequences(sequences, maxlen=50)

#tokenize test data

tokenizer.fit_on_texts(test_df['text'])
sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(sequences, maxlen=50)

# Model for Own Embeddings
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
training_metrics=model.fit(data, np.array(train_labels), validation_split=0.3, epochs=5)
score=model.evaluate(test_data, np.array(test_labels), verbose=2)

# Using pre-defined and pre-trained GloVe Embeddings

embeddings_index = dict()
f = open('Data/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            
#Model using Glove Embeddings

model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
training_metrics_glove=model_glove.fit(data, np.array(train_labels), validation_split=0.3, epochs=10)


score_glove=model_glove.evaluate(test_data, np.array(test_labels), verbose=2)

with open('Result.txt', 'a',encoding="utf8") as outfile:
    outfile.write("Training Accuracy for Network with own Embeddings: " +  str(training_metrics.history["acc"][-1])+"\n")
    outfile.write("Testing Accuracy for Network with own Embeddings: " +  str(score[1])+"\n")
  
    outfile.write("Training Accuracy for Network with GloVe Embeddings: " +  str(training_metrics_glove.history["acc"][-1])+"\n")  
    outfile.write("Testing Accuracy for Network with GloVe Embeddings: " +  str(score_glove[1])+"\n")

ModuleNotFoundError: No module named 'keras'